In [1]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password= secret_password)

from scipy.stats import ttest_ind

In [2]:
passe_data_json = client.get_in_possession_passes(params = {'competition_edition': 549, "group_by" : "team",
    "run_type" : ["run_in_behind", "run_ahead_of_the_ball", "support_run", "pulling_wide_run", "coming_short_run", "underlap_run",
                  "overlap_run", "dropping_off_run", "pulling_half_space_run", "cross_receiver_run"]})

In [3]:
ranking = ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"]

In [5]:
passe_data = pd.DataFrame(passe_data_json).set_index("team_name")
passe_data.drop(["team_id", "third", "channel", "minutes_played_per_match", "adjusted_min_tip_per_match",
                   "count_match", "count_match_failed", "count_opportunities_to_pass_to_runs_in_behind_in_sample"], inplace = True, axis = 1)
passe_data
passe_data = passe_data.reindex(ranking)
passe_data.to_excel("metrique_passe.xlsx", header = True, index = True)

In [6]:
top5 = ranking[:5]
top15 = ranking[5:]
top5_df = passe_data.loc[top5]
top15_df = passe_data.loc[top15]

In [7]:
dic_pvalue = {}

for i in passe_data.columns :
    ttest, pvalue = ttest_ind(top5_df[i], top15_df[i])
    dic_pvalue[i] = pvalue

c:\Users\fabie\Documents\Programmation\Github\Skill Corner-CF\env\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [8]:
tri = pd.DataFrame(pd.Series(dic_pvalue), columns=["pvalue"]).sort_values(by="pvalue")

In [9]:
tri.to_excel("student_passe.xlsx")